In [1]:
import pandas as pd
import sklearn as sk
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import statistics


In [2]:
data = pd.read_csv('cars_data_no_encoded.csv')

In [3]:
columns = list(data.columns)
columns.remove('price')


In [4]:
data_cols = data
data_cols = data_cols.sample(20000, random_state= 42)

In [5]:
#data_cols = data

In [6]:
data_cols.back_legroom.fillna(0, inplace = True)
#data_cols.bed.fillna('no_bed',inplace= True)
#data_cols.bed_height.fillna(0,inplace= True)
#data_cols.bed_length.fillna(0 ,inplace= True)
data_cols.seller_rating.fillna(0, inplace= True)



In [7]:
data_cols.tail()

,back_legroom,body_type,city,city_fuel_economy,daysonmarket,engine_displacement,engine_type,franchise_dealer,front_legroom,fuel_type,...,price,seller_rating,torque,transmission,wheel_system_display,wheelbase,width,year,word_count_description,extras
99602,37.4,Sedan,Other,29.0,56,1800.0,I4,True,42.5,Gasoline,...,14882.0,3.714286,125.0,CVT,Front-Wheel Drive,106.3,69.3,2019,578,5
82479,0.0,Pickup Truck,Other,15.0,62,5700.0,V8,True,NaN,Gasoline,...,36995.0,4.736842,NaN,A,Four-Wheel Drive,NaN,NaN,2019,611,3
130314,38.2,SUV / Crossover,Other,21.0,62,2400.0,I4,True,41.5,Gasoline,...,20757.0,4.538462,175.0,A,All-Wheel Drive,105.1,73.0,2017,652,9
68790,38.0,Sedan,Other,20.0,39,3600.0,V6,True,42.0,Gasoline,...,26480.0,4.458333,268.0,A,Front-Wheel Drive,114.4,73.5,2019,305,5
33687,38.6,SUV / Crossover,Other,18.0,14,3600.0,V6,True,40.3,Gasoline,...,34534.0,4.600000,260.0,A,Four-Wheel Drive,114.8,84.8,2018,772,5


In [8]:
data_cols.describe()

,back_legroom,city_fuel_economy,daysonmarket,engine_displacement,front_legroom,height,horsepower,maximum_seating,mileage,price,seller_rating,torque,wheelbase,width,year,word_count_description,extras
count,20000.000000,19513.000000,20000.000000,19867.000000,18991.000000,19072.000000,19136.000000,19073.000000,19794.000000,20000.000000,20000.000000,19136.000000,19073.000000,19071.000000,20000.000000,20000.000000,20000.000000
mean,34.835860,21.987342,52.934550,3027.331756,42.255626,65.009501,243.441942,5.464269,58504.663686,21602.697487,4.216557,258.053115,114.164138,77.713492,2015.524300,456.477400,4.458750
std,10.094512,8.903940,86.376899,1286.774935,1.581883,7.213263,82.637175,1.102161,46696.362521,11555.824204,0.747870,107.260158,13.131012,7.320108,4.691419,263.426594,2.862273
min,0.000000,9.000000,0.000000,700.000000,35.800000,43.900000,70.000000,2.000000,50.000000,900.000000,0.000000,68.000000,73.500000,61.400000,1932.000000,0.000000,0.000000
25%,35.300000,17.000000,12.000000,2000.000000,41.200000,57.900000,175.000000,5.000000,25470.250000,13495.000000,4.000000,178.000000,106.300000,72.400000,2014.000000,264.000000,2.000000
50%,37.400000,21.000000,28.000000,2500.000000,42.000000,66.000000,244.000000,5.000000,41034.500000,18999.000000,4.347826,255.000000,110.500000,75.800000,2017.000000,436.000000,4.000000
75%,39.200000,25.000000,56.000000,3600.000000,43.000000,69.900000,295.000000,6.000000,84111.250000,27998.000000,4.619048,280.000000,116.000000,81.800000,2018.000000,622.000000,7.000000
max,47.500000,127.000000,2730.000000,8100.000000,67.000000,111.300000,600.000000,12.000000,357564.000000,67000.000000,5.000000,935.000000,201.000000,105.900000,2021.000000,2221.000000,12.000000


In [9]:
data_cols[data_cols.front_legroom.isna()]
data_cols = data_cols[data_cols['front_legroom'].notna()]


In [10]:
data_cols['avg_miles_per_year'] = (2021 - data_cols['year']) * data_cols['mileage']

In [11]:
data_cols['avg_miles_per_year'].mean()

433680.3311004785

In [12]:
# calcular city fuel economy
def cfe(row, df):
    if pd.isna(row['city_fuel_economy']):
        mean_value = df.groupby('model_name')['city_fuel_economy'].mean()[row['model_name']]
        return mean_value
    else:
        return row['city_fuel_economy']

# calcular engine displacement
def engine_disp(row, df):
    if pd.isna(row['engine_displacement']):
        mean_value = df.groupby('model_name')['engine_displacement'].mean()[row['model_name']]
        return mean_value
    else:
        return row['engine_displacement']
    
# calcular highway fuel economy
    
def hwfe(row, df):
    if pd.isna(row['highway_fuel_economy']):
        mean_value = df.groupby('model_name')['highway_fuel_economy'].mean()[row['model_name']]
        return mean_value
    else:
        return row['city_fuel_economy']
    
# calcular horse power    
def hp(row, df):
    if pd.isna(row['horsepower']):
        mean_value = df.groupby('model_name')['horsepower'].mean()[row['model_name']]
        return mean_value
    else:
        return row['horsepower']
    
# calculate torque    
def calculate_torque(row, df):
    if pd.isna(row['torque']):
        torque = (row['horsepower'] * 5252) / 4000
        return torque
    else:
        return row['torque']
    




In [13]:
avg_yearly = data_cols['avg_miles_per_year'].mean()
avg_yearly

433680.3311004785

In [14]:
def calculate_mileage(row, df):  
    if pd.isna(row['mileage']):
        mileage = 2021 - row['year'] * 233897.81
        return mileage
    else:
        return row['mileage']

In [15]:
data_cols['city_fuel_economy'] = data_cols.apply(cfe, args=(data_cols,), axis=1)
data_cols['engine_displacement'] = data_cols.apply(engine_disp, args=(data_cols,), axis=1)
#data_cols['highway_fuel_economy'] = data_cols.apply(hwfe, args=(data_cols,), axis=1)
data_cols['horsepower'] = data_cols.apply(hp, args=(data_cols,), axis=1)
data_cols['torque'] = data_cols.apply(calculate_torque, args=(data_cols,), axis=1)
data_cols['mileage'] = data_cols.apply(calculate_mileage, args=(data_cols,), axis=1)

In [16]:
data.head()
data['year'].unique()

array([2019, 2017, 2010, 2014, 2011, 2020, 2012, 2008, 2009, 2018, 2016,
       2002, 2015, 2013, 2006, 2004, 2007, 2003, 1966, 1993, 2005, 1995,
       1986, 2001, 1999, 1969, 1985, 2000, 1967, 1979, 1984, 1998, 2021,
       1991, 1951, 1974, 1950, 1973, 1970, 1982, 1996, 1989, 1968, 1992,
       1994, 1961, 1964, 1978, 1976, 1997, 1952, 1988, 1987, 1963, 1957,
       1983, 1965, 1971, 1980, 1960, 1931, 1948, 1955, 1962, 1938, 1972,
       1953, 1990, 1981, 1975, 1936, 1941, 1932, 1956, 1915, 1977, 1959,
       1947, 1954, 1930, 1923, 1928, 1940, 1958, 1949])

In [17]:
data_cols.describe()

,back_legroom,city_fuel_economy,daysonmarket,engine_displacement,front_legroom,height,horsepower,maximum_seating,mileage,price,seller_rating,torque,wheelbase,width,year,word_count_description,extras,avg_miles_per_year
count,18991.000000,18991.000000,18991.000000,18991.000000,18991.000000,18991.000000,18991.000000,18991.000000,1.899100e+04,18991.000000,18991.000000,18991.000000,18991.000000,18991.000000,18991.000000,18991.000000,18991.000000,1.881000e+04
mean,36.686704,21.527683,51.815702,3006.276657,42.255626,65.011911,243.284481,5.468274,-4.436884e+06,21544.669409,4.216291,257.765941,114.157922,77.700632,2015.791691,462.553578,4.543468,4.336803e+05
std,6.277615,5.552210,79.647824,1273.112638,1.581883,7.179037,82.539708,1.098868,4.581923e+07,11368.385294,0.740268,107.327763,13.126849,7.323058,3.739785,261.501537,2.843485,6.339356e+05
min,0.000000,9.000000,0.000000,700.000000,35.800000,43.900000,70.000000,2.000000,-4.724716e+08,900.000000,0.000000,68.000000,73.500000,61.400000,1986.000000,0.000000,0.000000,0.000000e+00
25%,35.700000,17.000000,12.000000,2000.000000,41.200000,57.900000,175.000000,5.000000,2.485400e+04,13658.000000,4.000000,177.000000,106.300000,72.400000,2014.000000,274.000000,2.000000,7.001175e+04
50%,37.600000,21.000000,28.000000,2500.000000,42.000000,66.000000,244.000000,5.000000,4.025300e+04,18999.000000,4.342857,255.000000,110.500000,75.800000,2017.000000,443.000000,4.000000,1.509720e+05
75%,39.400000,25.000000,56.000000,3600.000000,43.000000,69.900000,295.000000,6.000000,8.208250e+04,27990.000000,4.615385,280.000000,116.000000,81.800000,2018.000000,627.000000,7.000000,5.400332e+05
max,47.500000,57.000000,1747.000000,8100.000000,67.000000,111.300000,577.000000,12.000000,3.575640e+05,67000.000000,5.000000,935.000000,201.000000,105.900000,2021.000000,2144.000000,12.000000,6.986973e+06


In [18]:
del data_cols['avg_miles_per_year']
data_cols.dropna(inplace = True)

In [19]:
data_cols.isna().sum()

back_legroom              0
body_type                 0
city                      0
city_fuel_economy         0
daysonmarket              0
engine_displacement       0
engine_type               0
franchise_dealer          0
front_legroom             0
fuel_type                 0
height                    0
horsepower                0
listing_color             0
make_name                 0
maximum_seating           0
mileage                   0
model_name                0
price                     0
seller_rating             0
torque                    0
transmission              0
wheel_system_display      0
wheelbase                 0
width                     0
year                      0
word_count_description    0
extras                    0
dtype: int64

In [20]:
data_cols.shape

(18446, 27)

In [21]:
data_cols.to_csv('cars_clean.csv')

In [22]:
def confidence(data, alpha):
    import statistics
    import scipy.stats as stats


    # Calculate sample mean and standard deviation
    sample_mean = statistics.mean(data)
    sample_std = statistics.stdev(data)

    # Sample size
    n = len(data)

    # Desired level of confidence (95%)
    confidence = alpha

    # Calculate the critical value (t-distribution)
    critical_value = stats.t.ppf((1 + confidence) / 2, df=n-1)

    # Calculate the margin of error
    margin_of_error = critical_value * (sample_std / (n ** 0.5))

    # Calculate the confidence interval
    confidence_interval = (sample_mean - margin_of_error, sample_mean + margin_of_error)

    return (sample_mean, confidence_interval)

    # print("Sample mean:", sample_mean)
    # print("Margin of error:", margin_of_error)
    # print("Confidence interval:", confidence_interval)


## Recomendation system

In [23]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import pairwise_distances

# Load data
data = pd.read_csv('cars_clean.csv')

# Select features and target variable

#X = data[columns]
X = data[['engine_displacement','horsepower','mileage','make_name','model_name', 'year' ]]

y = data['price']


# Convert categorical variables into numerical ones using one-hot encoding
#X = pd.get_dummies(X,drop_first= True)


# Define a function to compute the pairwise distances between all pairs of cars
def get_distance_matrix(X, metric='euclidean'):
    if metric == 'cosine':
        return pairwise_distances(X, metric='cosine')
    elif metric == 'euclidean':
        return pairwise_distances(X, metric='euclidean')
    elif metric == 'manhattan':
        return pairwise_distances(X, metric='manhattan')
    elif metric == 'chebyshev':
        return pairwise_distances(X, metric='chebyshev')
    else:
        raise ValueError('Invalid distance metric')

# Define a function to get the top k recommendations for a given car
def get_recommendations(car_index, k, J, distance_metric):
    # Get the distance scores of the car with other cars
    distance_scores = list(enumerate(get_distance_matrix(X, metric=distance_metric)[car_index]))
    # Sort the cars based on their distance scores
    distance_scores = sorted(distance_scores, key=lambda x: x[1])
    # Get the indices of the top k similar cars
    top_indices = [i[0] for i in distance_scores[1:k+1]]
    
    # Return top J indices and generate an interval
    top_J = [i[0] for i in distance_scores[1:J+1]]
    

    
    top_prices_k = data.iloc[top_indices]['price']
    top_prices_j = data.iloc[top_J]['price']

    max_price = max(top_prices_j)
    min_price = min(top_prices_j)

    confidence_intervals = confidence(top_prices_j, 0.95)
    
    # Return the top k recommendations
    
    return top_prices_k, max_price, min_price, confidence_intervals, top_prices_j
    #return data.iloc[top_indices]['price'], max_price, min_price, confidence_intervals, top_indices, top_J





In [24]:
X.tail()

,engine_displacement,horsepower,mileage,make_name,model_name,year
18441,2000.0,184.0,17120.0,Volkswagen,Other,2019
18442,1800.0,124.0,40971.0,Nissan,Sentra,2019
18443,2400.0,181.0,51977.0,Kia,Other,2017
18444,3600.0,310.0,7726.0,Buick,Other,2019
18445,3600.0,295.0,23521.0,Jeep,Grand Cherokee,2018


In [25]:
X2 = X.copy()

## Datos de prueba

In [26]:
new_cars = [{'make_name': ['Ford'], 'model_name': ['F-150'], 'year': [2015], 'mileage': [35000], 'engine_displacement': [5200], 'horsepower':[320]},
            {'make_name': ['Hyundai'], 'model_name': ['Elantra'], 'year': [2000], 'mileage': [250000], 'engine_displacement': [1500], 'horsepower':[90]},
            {'make_name': ['Honda'], 'model_name': ['CR-V'], 'year': [2018], 'mileage': [30000], 'engine_displacement': [2500], 'horsepower':[160]},
            {'make_name': ['Ford'], 'model_name': ['Fusion'], 'year': [2020], 'mileage': [300000], 'engine_displacement': [3000], 'horsepower':[250]},
            {'make_name': ['Jeep'], 'model_name': ['Grand Caravan'], 'year': [1999], 'mileage': [222000], 'engine_displacement': [5200], 'horsepower':[250]},
            {'make_name': ['Chevrolet'], 'model_name': ['Silverado 1500'], 'year': [2000], 'mileage': [15000], 'engine_displacement': [4500], 'horsepower':[300]},
            {'make_name': ['Nissan'], 'model_name': ['Altima'], 'year': [2010], 'mileage': [95000], 'engine_displacement': [2500], 'horsepower':[160]},
            {'make_name': ['Toyota'], 'model_name': ['RAV4'], 'year': [2011], 'mileage': [105000], 'engine_displacement': [2000], 'horsepower':[150]},
            {'make_name': ['Nissan'], 'model_name': ['Sentra'], 'year': [2019], 'mileage': [100000], 'engine_displacement': [1800], 'horsepower':[130]},
            {'make_name': ['Jeep'], 'model_name': ['Grand Cherokee'], 'year': [2015], 'mileage': [30000], 'engine_displacement': [4000], 'horsepower':[280]},
            {'make_name': ['Honda'], 'model_name': ['Civic'], 'year': [2013], 'mileage': [3500], 'engine_displacement': [2000], 'horsepower':[200]},
            {'make_name': ['Honda'], 'model_name': ['Accord'], 'year': [2012], 'mileage': [32000], 'engine_displacement': [2500], 'horsepower':[250]},
            {'make_name': ['Chevrolet'], 'model_name': ['Equinox'], 'year': [2016], 'mileage': [85000], 'engine_displacement': [3000], 'horsepower':[180]},
            {'make_name': ['Other'], 'model_name': ['Other'], 'year': [2018], 'mileage': [95000], 'engine_displacement': [1500], 'horsepower':[100]},
            {'make_name': ['Toyota'], 'model_name': ['Camry'], 'year': [2019], 'mileage': [65000], 'engine_displacement': [2500], 'horsepower':[180]},
            {'make_name': ['Ford'], 'model_name': ['Edge'], 'year': [2019], 'mileage': [34000], 'engine_displacement': [5200], 'horsepower':[320]},
            {'make_name': ['Chevrolet'], 'model_name': ['Malibu'], 'year': [2017], 'mileage': [235000], 'engine_displacement': [3500], 'horsepower':[200]},
            {'make_name': ['Toyota'], 'model_name': ['Corolla'], 'year': [2016], 'mileage': [135000], 'engine_displacement': [1800], 'horsepower':[120]},
            {'make_name': ['Ford'], 'model_name': ['F-150'], 'year': [2011], 'mileage': [35000], 'engine_displacement': [5200], 'horsepower':[320]},
            {'make_name': ['Hyundai'], 'model_name': ['Elantra'], 'year': [2007], 'mileage': [25000], 'engine_displacement': [2000], 'horsepower':[100]},
            {'make_name': ['Honda'], 'model_name': ['CR-V'], 'year': [2012], 'mileage': [70000], 'engine_displacement': [2500], 'horsepower':[160]},
            {'make_name': ['Ford'], 'model_name': ['Fusion'], 'year': [2020], 'mileage': [200000], 'engine_displacement': [3000], 'horsepower':[250]},
            {'make_name': ['Jeep'], 'model_name': ['Grand Caravan'], 'year': [1999], 'mileage': [122000], 'engine_displacement': [5200], 'horsepower':[250]},
            {'make_name': ['Chevrolet'], 'model_name': ['Silverado 1500'], 'year': [2000], 'mileage': [150000], 'engine_displacement': [4500], 'horsepower':[300]},
            {'make_name': ['Nissan'], 'model_name': ['Altima'], 'year': [2010], 'mileage': [5000], 'engine_displacement': [2500], 'horsepower':[160]},
            {'make_name': ['Toyota'], 'model_name': ['RAV4'], 'year': [2011], 'mileage': [705000], 'engine_displacement': [2000], 'horsepower':[150]},
            {'make_name': ['Nissan'], 'model_name': ['Sentra'], 'year': [2019], 'mileage': [700000], 'engine_displacement': [1800], 'horsepower':[130]},
            {'make_name': ['Jeep'], 'model_name': ['Grand Cherokee'], 'year': [2015], 'mileage': [300000], 'engine_displacement': [4000], 'horsepower':[280]},
            {'make_name': ['Honda'], 'model_name': ['Civic'], 'year': [2013], 'mileage': [3500], 'engine_displacement': [2000], 'horsepower':[200]},
            {'make_name': ['Honda'], 'model_name': ['Accord'], 'year': [2012], 'mileage': [32000], 'engine_displacement': [2500], 'horsepower':[250]},
            {'make_name': ['Chevrolet'], 'model_name': ['Equinox'], 'year': [2016], 'mileage': [85000], 'engine_displacement': [3000], 'horsepower':[180]},
            {'make_name': ['Other'], 'model_name': ['Other'], 'year': [2018], 'mileage': [95000], 'engine_displacement': [1500], 'horsepower':[100]},
            {'make_name': ['Toyota'], 'model_name': ['Camry'], 'year': [2019], 'mileage': [65000], 'engine_displacement': [2500], 'horsepower':[180]},
            {'make_name': ['Ford'], 'model_name': ['Edge'], 'year': [2009], 'mileage': [34000], 'engine_displacement': [5200], 'horsepower':[320]},
            {'make_name': ['Chevrolet'], 'model_name': ['Malibu'], 'year': [2010], 'mileage': [235000], 'engine_displacement': [3500], 'horsepower':[200]},
            {'make_name': ['Toyota'], 'model_name': ['Corolla'], 'year': [2006], 'mileage': [135000], 'engine_displacement': [1800], 'horsepower':[120]},{'make_name': ['Ford'], 'model_name': ['F-150'], 'year': [2015], 'mileage': [35000], 'engine_displacement': [5200], 'horsepower':[320]},
            {'make_name': ['Hyundai'], 'model_name': ['Elantra'], 'year': [2017], 'mileage': [250000], 'engine_displacement': [2000], 'horsepower':[100]},
            {'make_name': ['Honda'], 'model_name': ['CR-V'], 'year': [2015], 'mileage': [10000], 'engine_displacement': [2500], 'horsepower':[160]},
            {'make_name': ['Ford'], 'model_name': ['Fusion'], 'year': [2010], 'mileage': [300000], 'engine_displacement': [3000], 'horsepower':[250]},
            {'make_name': ['Jeep'], 'model_name': ['Grand Caravan'], 'year': [2000], 'mileage': [22000], 'engine_displacement': [5200], 'horsepower':[250]},
            {'make_name': ['Chevrolet'], 'model_name': ['Silverado 1500'], 'year': [2020], 'mileage': [15000], 'engine_displacement': [4500], 'horsepower':[300]},
            {'make_name': ['Nissan'], 'model_name': ['Altima'], 'year': [2010], 'mileage': [9500], 'engine_displacement': [2500], 'horsepower':[160]},
            {'make_name': ['Toyota'], 'model_name': ['RAV4'], 'year': [209], 'mileage': [10500], 'engine_displacement': [2000], 'horsepower':[150]},
            {'make_name': ['Nissan'], 'model_name': ['Sentra'], 'year': [2025], 'mileage': [150000], 'engine_displacement': [1800], 'horsepower':[130]},
            {'make_name': ['Jeep'], 'model_name': ['Grand Cherokee'], 'year': [2015], 'mileage': [30000], 'engine_displacement': [4000], 'horsepower':[280]},
            {'make_name': ['Honda'], 'model_name': ['Civic'], 'year': [2013], 'mileage': [50000], 'engine_displacement': [2000], 'horsepower':[200]},
            {'make_name': ['Honda'], 'model_name': ['Accord'], 'year': [2010], 'mileage': [320000], 'engine_displacement': [2500], 'horsepower':[250]},
            {'make_name': ['Chevrolet'], 'model_name': ['Equinox'], 'year': [2016], 'mileage': [85000], 'engine_displacement': [3000], 'horsepower':[180]},
            {'make_name': ['Other'], 'model_name': ['Other'], 'year': [2008], 'mileage': [95000], 'engine_displacement': [1500], 'horsepower':[100]},
            {'make_name': ['Toyota'], 'model_name': ['Camry'], 'year': [2009], 'mileage': [85000], 'engine_displacement': [2500], 'horsepower':[180]},
            {'make_name': ['Ford'], 'model_name': ['Edge'], 'year': [2009], 'mileage': [340000], 'engine_displacement': [5200], 'horsepower':[320]},
            {'make_name': ['Chevrolet'], 'model_name': ['Malibu'], 'year': [2017], 'mileage': [235000], 'engine_displacement': [3500], 'horsepower':[200]},
            {'make_name': ['Toyota'], 'model_name': ['Corolla'], 'year': [2004], 'mileage': [235000], 'engine_displacement': [1800], 'horsepower':[120]}
]

## Recomendador
solo es necesario llenar la variable new_vehicle reemplazando los valores.

In [27]:
# Se deja este codigo comentado pues si se ejecuta afecta las ejecuciones posteriores 
# sin embargo este codigo muestra los resultados si un usuario regular hiciera solicitud
# el codigo siguiente ejecuta las recomendaciones para variable new_cars


# #Recomendador
# #Ingresar nuevo vehiculo
# X = X2
# new_vehicle = pd.DataFrame(new_cars[13])
# print(new_cars[13])

# # se añade el vehiculo a la matriz del modelo 
# new_vehicle = new_vehicle.fillna(0)
# X = pd.concat([X, new_vehicle], ignore_index=True)
# X = pd.get_dummies(X, dummy_na=True, drop_first=True)
# X = X.fillna(0)



# # se generan los datos necesarios para la ejecucion del recomendador
# car_index = len(X) - 1 # index of the new vehicle in the similarity matrix
# k = 5 # numero de recomendaciones
# J = 100 # numero de proxies

# distance_metric = 'euclidean' # metrica a utilizar la euclideana fue la que obtuvo mejor rendimiento
# #generamos las recomendaciones
# recommendations, max_p, min_p, confidence_intervals, recsK = get_recommendations(car_index, k, J, distance_metric)
# #recomendamos un precio
# recomended_price = statistics.mean(recommendations)

# print(f'closest prices go from minimum: {min_p}, to maximum: {max_p}. we recomend {recomended_price}')

# print()

# print(f'the price confidence intervals are {confidence_intervals[1][0]} and {confidence_intervals[1][1]}, the recomended price is {recomended_price}, mean price {confidence_intervals[0]}')



In [ ]:
preds = {
    'model': [],
    'year': [],
    'price': [],
    'mean_price': [],
    'confidence_interval': []
}

for i in new_cars:
    X = X2
    new_vehicle = pd.DataFrame(i)

    # se añade el vehiculo a la matriz del modelo 
    new_vehicle = new_vehicle.fillna(0)
    X = pd.concat([X, new_vehicle], ignore_index=True)
    X = pd.get_dummies(X, dummy_na=True, drop_first=True)
    X = X.fillna(0)



    # se generan los datos necesarios para la ejecucion del recomendador
    car_index = len(X) - 1 # indice del ultimo vehiculo ingresado (para el que vamos a recomendar)
    k = 5 # numero de recomendaciones
    J = 100 # numero de proxies

    distance_metric = 'euclidean' # metrica a utilizar la euclideana fue la que obtuvo mejor rendimiento
    #generamos las recomendaciones
    recommendations, max_p, min_p, confidence_intervals, recsK = get_recommendations(car_index, k, J, distance_metric)
    #recomendamos un precio
    recomended_price = statistics.mean(recommendations)


    price = recomended_price
    ci = confidence_intervals
    
    
    name = i['model_name'][0]
    year = i['year'][0]
    preds['model'].append(name)
    preds['year'].append(year)
    preds['price'].append(price)
    preds['mean_price'].append(ci[0])
    preds['confidence_interval'].append(ci[1])



In [29]:
pd.DataFrame(preds).head(10)

,model,year,price,mean_price,confidence_interval
0,F-150,2015,37602.2,36974.08,"(35299.60616978301, 38648.553830216995)"
1,Elantra,2000,9006.8,7763.35,"(6608.922988219222, 8917.777011780778)"
2,CR-V,2018,20715.6,20617.16,"(19609.773772513014, 21624.546227486986)"
3,Fusion,2020,10895.8,7689.75,"(6542.858441787637, 8836.641558212363)"
4,Grand Caravan,1999,5957.0,7266.98,"(6233.900075123513, 8300.059924876487)"
5,Silverado 1500,2000,36633.6,35580.07,"(33914.65140798328, 37245.48859201672)"
6,Altima,2010,12375.2,12301.74,"(11515.565771154954, 13087.914228845046)"
7,RAV4,2011,10153.6,10286.66,"(9625.499690381628, 10947.820309618372)"
8,Sentra,2019,8784.4,10900.74,"(10118.457460812748, 11683.022539187252)"
9,Grand Cherokee,2015,33201.0,28584.24,"(27145.36861059491, 30023.111389405094)"


In [30]:
# Append new vehicle to the dataset
all_recs = []
for i in range(0, len(new_cars)):
    X = X2.copy()
    new_vehicle = pd.DataFrame(new_cars[i])
    new_vehicle = new_vehicle.fillna(0)
    X = pd.concat([X, new_vehicle], ignore_index=True)
    X = pd.get_dummies(X, dummy_na=True, drop_first=True)
    X = X.fillna(0)
    recs = []
    for i in ['euclidean', 'cosine', 'manhattan', 'chebyshev']:
        import statistics
        car_index = len(X) - 1 # index of the new vehicle in the similarity matrix
        k = 5 # number of recommendations
        J = 100 # number of proxies for variance
        distance_metric = i # distance metric to use
        recommendations, max_p, min_p, confidence_intervals, recsK = get_recommendations(car_index, k, J, distance_metric)
        recomended_price = statistics.mean(recommendations)
        recs.append((i, recomended_price, confidence_intervals))
    all_recs.append(recs)


## Testing

In [31]:
car_stats = []
for i in range(54):
    car_stats.append((new_cars[i]['model_name'][0], new_cars[i]['year'][0], all_recs[i]))
     

In [32]:
intervalos = {
    'euclidea': [],
    'coseno': [],
    'manhattan': [],
    'chebyshev': []
}

for i in range(54):
    euclidea  = car_stats[i][2][0][2][1][1] - car_stats[i][2][0][2][1][0]
    coseno =car_stats[i][2][1][2][1][1] - car_stats[i][2][1][2][1][0]
    manhattan  =car_stats[i][2][2][2][1][1] - car_stats[i][2][2][2][1][0]
    chebyshev = car_stats[i][2][3][2][1][1] - car_stats[i][2][3][2][1][0]

    intervalos['euclidea'].append(euclidea)
    intervalos['coseno'].append(coseno)
    intervalos['manhattan'].append(manhattan)
    intervalos['chebyshev'].append(chebyshev)


In [33]:
intervalos_df = pd.DataFrame(intervalos)
euc = intervalos_df['euclidea'].mean()
cos = intervalos_df['coseno'].mean()
man = intervalos_df['manhattan'].mean()
cheb = intervalos_df['chebyshev'].mean()

print(f'euclidea: {euc}\ncoseno: {cos}\nmanhattan{man}\nchebyshev : {cheb}')

euclidea: 2483.814034192585
coseno: 1911.4849528964712
manhattan2486.288489116306
chebyshev : 2468.227161085973


In [34]:
euclidean = []
cos = []
man = []
cheb =[]
mean_price = []
car_model = []
car_year = []

for i in car_stats:
    car = i[0]
    year = i[1]

    mean_car_value = data[(data['model_name'] == car) & (data['year'] == year)]['price'].mean()
    euclidea = i[2][0][1]
    coseno = i[2][1][1]
    manhattan = i[2][2][1]
    chebyshev = i[2][3][1]
    
    euclidean.append(euclidea)
    cos.append(coseno)
    man.append(manhattan)
    cheb.append(chebyshev)
    mean_price.append(mean_car_value)
    car_model.append(car)
    car_year.append(year)

allcars = {'modelo': car_model, 'año': car_year, 'precio medio': mean_price,
            'euclidiana': euclidean, 'coseno': cos, 'manhattan': man, 'chebyshev':cheb }

cars_df = pd.DataFrame(allcars)


## Calculo de las differencias entre el precio medio y las distintas distancias

In [35]:
cars_df['euclidea diff'] = abs(cars_df['precio medio'] - cars_df['euclidiana'])
cars_df['coseno diff'] = abs(cars_df['precio medio'] - cars_df['coseno'])
cars_df['manhattan diff'] = abs(cars_df['precio medio'] - cars_df['manhattan'])
cars_df['chebyshev diff'] = abs(cars_df['precio medio'] - cars_df['chebyshev'])
cars_df.head()
cars_df = cars_df.dropna(axis = 0)


In [36]:
cars_df.head()

,modelo,año,precio medio,euclidiana,coseno,manhattan,chebyshev,euclidea diff,coseno diff,manhattan diff,chebyshev diff
0,F-150,2015,29037.909091,37602.2,38563.4,37602.2,38037.0,8564.290909,9525.490909,8564.290909,8999.090909
2,CR-V,2018,25633.675000,20715.6,20503.6,20715.6,20715.6,4918.075000,5130.075000,4918.075000,4918.075000
3,Fusion,2020,22431.000000,10895.8,2882.8,10895.8,10895.8,11535.200000,19548.200000,11535.200000,11535.200000
6,Altima,2010,7990.000000,12375.2,8817.0,11754.2,14088.0,4385.200000,827.000000,3764.200000,6098.000000
7,RAV4,2011,10791.125000,10153.6,7853.2,10153.6,9747.0,637.525000,2937.925000,637.525000,1044.125000


## distancias medias y medianas de la muestra de pruebas

In [37]:
print('mean')
print(f'euclidea: {cars_df["euclidea diff"].mean()}')
print(f'coseno: {cars_df["coseno diff"].mean()}')
print(f'manhattan: {cars_df["manhattan diff"].mean()}')
print(f'chebyshev: {cars_df["chebyshev diff"].mean()}')
print('\nmedian')
print(f'euclidea: {cars_df["euclidea diff"].median()}')
print(f'coseno: {cars_df["coseno diff"].median()}')
print(f'manhattan: {cars_df["manhattan diff"].median()}')
print(f'chebyshev: {cars_df["chebyshev diff"].median()}')

mean
euclidea: 9401.088005018164
coseno: 9811.174961539904
manhattan: 9101.705396322512
chebyshev: 9215.709744148597

median
euclidea: 7273.889189189191
coseno: 8205.335070785071
manhattan: 7278.144594594595
chebyshev: 6697.68918918919
